# Récupération des données

In [75]:
import pandas as pd
from math import isnan
import time 

In [2]:
df = pd.read_excel("PS_INFO.xlsx")

In [51]:
FR_Oral = df["Note à l'épreuve de Oral de Français (épreuve anticipée)"]
FR_Ecrit = df["Note à l'épreuve de Ecrit de Français (épreuve anticipée)"]

In [85]:
start = time.time()
for i in range(4600):
    a = recupDBC(df,i)
end = time.time()
elapsed = end - start
print(elapsed)

7.004727840423584


In [43]:
# Es-ce que le candidats à son Bac de Français ?
def BAC_FR(i):
    if (isnan(FR_Oral[i]) and isnan(FR_Ecrit[i])):
        return "NON"
    else :
        return "OUI"

In [84]:
def recupDBC(df,i):
    #Création de matière qui stock les notes du candidat
    matiere = pd.DataFrame()
    #Création d'une DataFrame pour le Candidats i
    DBC ={
        'Validation':0,
        'Num_Can':df['ID'][i],
        'SEXE':df['Sexe'][i], 
        'FILIERE':'?',
        'Autre_Form':'?',
        'An_Bac': None,
        'Bourse': False, 
        'Nom_Eta' : df["Libellé établissement"][i], 
        'Com_Eta' : df["Commune établissement"][i], 
        'Depa_Eta' : df["Département établissement"][i]
    }
    
    
    #Récupération des informations:
    #Quelle est sa formations scolaire précédent sa candidatures
    Form = df['Série diplôme'][i]
    
    
    #Prise en compte des notes de BAC de Français
    FR_Oral = df["Note à l'épreuve de Oral de Français (épreuve anticipée)"][i]
    FR_Ecrit = df["Note à l'épreuve de Ecrit de Français (épreuve anticipée)"][i]   
    
    
    #Le Candidats est-il boursier ?
    if(df["Boursier"][i] in ["Boursier de l'enseignement supérieur", 'Boursier du secondaire']) : 
        DBC['BOURSIER'] = True # Mise à jour de sa situation de bourse
        
    #Es-ce que le candidats est en formation Générale ou Technologique ?    
        DBC['FILIERE']='Générale'
    else : 
        DBC['FILIERE']='Technologique'
        
    
    #On vas attrivuer des points:
    # -1 vaut une élémination
    #  0 sa candidatures sera prise en compte en manuel
    #  1 sa candidatures vas être 
    if(df['Candidature validée (O/N)'][i]=='NON') : 
        # Si non validé éléminé
        DBC['Validation']= -1    
        
    # On recherche les DAEU Mathématiques.
    # Dans un premier temps nous n'avons que les noms en DAEU
    elif Form == "DAEU" : 
        # On recherche plus précisemment les Daeu-b qui sont les DAEU MAthématiques
        if df["Spécialité diplôme bac Pro et anciens bacs"][i] == "Daeu-b" :
            # Dossier manuel
            DBC['Validation']=0
            DBC['Autre_Form']="Daeu-b"
            return DBC
        else:
            # Si DAEU-a, donc Littérature éléminé
            DBC['Validation']=-1
            
    # On recherche les anciens Bac ou Bac Internationale
    elif(Form in ["Bac général de plus de 15 ans","Baccalauréat International"]) : 
        #Bac en cours mais pas classique => dossier à étudier à la main
        DBC['Validation']=0
        DBC['Autre_Form']="AUTRES"
        return DBC
    
    # On cherche les Bac Scientifiques, qui ont validé leur Bac de français
    elif(BAC_FR(i)=="OUI" and Form=="Scientifique") : 
        DBC["Validation"]=1
        #Math
        matiere['MATH']={
            'NOM' : 'Mathématiques',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }
        
        #Français
        matiere['FRANCAIS']={
            'NOM' : 'Français',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        } 
        
        #Spécialité
        SPE=df["Spécialité diplôme bac Pro et anciens bacs"][i]

        if SPE in ["Informatique et sciences du numérique","Mathématiques"]:
            matiere["SPE"]={
                'NOM' : SPE,
                'NOTE' : {
                    'P1':'Trimestre 1.1',
                    'P2':'Trimestre 2.1',
                    'P3':'Trimestre 3.1',
                    'T1':'Trimestre 1',
                    'T2':'Trimestre 2',
                    'T3':'Trimestre 3'
                }
            }
        
        #Anglais
        LV1=df["LV1"][i]
        LV2=df["LV2"][i]
        mat=None
        if(LV1=="Anglais") : mat="Langue vivante 1"
        if(LV2=="Anglais") : mat="Langue vivante 2"

        if(mat!=None) : 
            matiere['ANGLAIS']={
                'NOM' : mat,
                'NOTE' : {
                    'P1':'Trimestre 1.1',
                    'P2':'Trimestre 2.1',
                    'P3':'Trimestre 3.1',
                    'T1':'Trimestre 1',
                    'T2':'Trimestre 2',
                    'T3':'Trimestre 3'
                }
            }  
            
    # Recherche des Bac ES
    elif(BAC_FR(i)=="OUI" and Form=="Economique et social" ): 
        DBC["Validation"]=1
        #Math
        matiere['MATH']={
            'NOM' : 'Mathématiques',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }
        
        #Français
        matiere['FRANCAIS']={
            'NOM' : 'Français',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        } 
        
        #Spécialité
        SPE=df["Spécialité diplôme bac Pro et anciens bacs"][i]

        if SPE == "Mathématiques" :
            matiere[SPE]={
                'NOM' : SPE,
                'NOTE' : {
                    'P1':'Trimestre 1.1',
                    'P2':'Trimestre 2.1',
                    'P3':'Trimestre 3.1',
                    'T1':'Trimestre 1',
                    'T2':'Trimestre 2',
                    'T3':'Trimestre 3'
                }
            }
        
        #Anglais
        LV1=df["LV1"][i]
        LV2=df["LV2"][i]
        mat=None
        if(LV1=="Anglais") : mat="Langue vivante 1"
        if(LV2=="Anglais") : mat="Langue vivante 2"

        if(mat!=None) : 
            matiere['ANGLAIS']={
                'NOM' : mat,
                'NOTE' : {
                    'P1':'Trimestre 1.1',
                    'P2':'Trimestre 2.1',
                    'P3':'Trimestre 3.1',
                    'T1':'Trimestre 1',
                    'T2':'Trimestre 2',
                    'T3':'Trimestre 3'
                }
            }  
            
    # Recherche des candidats STI2D        
    elif(Form=="Sciences et Technologies de l'Industrie et du Développement Durable") : 
        
        DBC['Validation']=1
        
        #Sciences
        matiere['SCIENCES']={
            'NOM' : 'Physique-Chimie et Mathématiques',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }

        #Math
        matiere['MATH']={
            'NOM' : 'Mathématiques',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }

        #Informatique
        matiere['NSI']={
            'NOM' : "Systemes d''inForm et numerique",
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }
        
        #Français
        matiere['FRANCAIS']={
            'NOM' : 'Français',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }       

        #Anglais
        LV1=df["LV1"][i]
        LV2=df["LV2"][i]
        mat=None
        if(LV1=="Anglais") : mat="Langue vivante 1"
        if(LV2=="Anglais") : mat="Langue vivante 2"

        if(mat!=None) : 
            matiere['ANGLAIS']={
                'NOM' : mat,
                'NOTE' : {
                    'P1':'Trimestre 1.1',
                    'P2':'Trimestre 2.1',
                    'P3':'Trimestre 3.1',
                    'T1':'Trimestre 1',
                    'T2':'Trimestre 2',
                    'T3':'Trimestre 3'
                }
            }  
    #Recherche des candidats de Série Générale (nouveau bac)
    elif(Form=='Série Générale') : 
        
        DBC['Validation']=1

        #Sciences
        matiere['SCIENCES']={
            'NOM' : 'Enseignement scientifique',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }

        #Français
        matiere['FRANCAIS']={
            'NOM' : 'Français',
            'NOTE' : {
                'P1':'Trimestre 1.1',
                'P2':'Trimestre 2.1',
                'P3':'Trimestre 3.1',
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }       

        #Anglais
        LV1=df["LV1"][i]
        LV2=df["LV2"][i]
        mat=None
        if(LV1=="Anglais") : mat="Langue vivante 1"
        if(LV2=="Anglais") : mat="Langue vivante 2"

        if(mat!=None) : 
            matiere['ANGLAIS']={
                'NOM' : mat,
                'NOTE' : {
                    'P1':'Trimestre 1.1',
                    'P2':'Trimestre 2.1',
                    'P3':'Trimestre 3.1',
                    'T1':'Trimestre 1',
                    'T2':'Trimestre 2',
                'T3':'Trimestre 3'
                }
            }  

        #Enseignements de spécialités
        mat_interessante={
            "Mathématiques Spécialité":"MATH",
            "Numérique et Sciences Informatiques":"NSI"
        }

        #Les deux enseignements de spécialité de terminale
        mat1 = df["EDS BAC Terminale"][i]
        mat2 = df["EDS BAC Terminale.1"][i]

        for mat in [mat1, mat2] : 
            if(mat in mat_interessante.keys()) :
                matiere[mat_interessante[mat]]={
                    'NOM' : mat,
                    'NOTE' : {
                        'P1':'Trimestre 1.1',
                        'P2':'Trimestre 2.1',
                        'P3':'Trimestre 3.1',
                        'T1':'Trimestre 1',
                        'T2':'Trimestre 2',
                        'T3':'Trimestre 3'
                    }
                }   


        #Option Math complémentaire
        matiere['MATH_COMP']={
            'NOM' : 'Mathématiques Complémentaires',
            'NOTE' : {
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }

        #Option Math expertes
        matiere['MATH_EXP']={
            'NOM' : 'Mathématiques Expertes',
            'NOTE' : {
                'T1':'Trimestre 1',
                'T2':'Trimestre 2',
                'T3':'Trimestre 3'
            }
        }  
        
    #Si le candidats ne rentres dans aucune des conditions  
    else :
        DBC['Validation']=-1
        return DBC   
    return DBC   

Truc à integrer

In [ ]:
for mat in matiere : 
    trimestre=matiere[mat]['NOTE']
    DBC[mat]={}
    for tri in trimestre : 
        DBC[mat][tri] = recupNOTE(df, i, matiere[mat]['NOM'], trimestre[tri], a, b)